In [ ]:
from datetime import datetime, timedelta, timezone
import numpy as np

# occultation validity constraints
max_yaw = 65  # deg
range_elevation = (-200e3, 60e3)  # m
sample_elevation = 0  # m

# scenario configuration
start = datetime(2023, 12, 9, tzinfo=timezone.utc)
time_step = timedelta(seconds=10)
duration = timedelta(hours=1)
times = np.array([start + i * time_step for i in range(duration // time_step)])

In [ ]:
from tatc.schemas import Satellite, TwoLineElements

tx = Satellite(
    name="GPS BIIR-2  (PRN 13)", 
    orbit=TwoLineElements(
        tle=[
            "1 24876U 97035A   23352.79627337  .00000076  00000+0  00000+0 0  9991",
            "2 24876  55.6180 134.2234 0075558  53.5972 307.0755  2.00564787193676"
        ]
    ),
)

rx = Satellite(
    name="COSMIC-2 FM5",
    orbit=TwoLineElements(
        tle=[
            "1 44358U 19036V   23354.40324524  .00015027  00000-0  91237-3 0  9994",
            "2 44358  24.0023   0.0734 0003537 251.0816 108.9304 15.08623759245240",
        ]
    ),
)

from tatc.analysis import collect_ro_observations

ro_obs = collect_ro_observations(
    rx, tx, times, sample_elevation, max_yaw, range_elevation
)
display(ro_obs)

In [ ]:
import numpy as np
from skyfield.api import wgs84, Distance, load
from skyfield.positionlib import Geocentric

timescale = load.timescale()

def _valid_ro(tx, rx, max_yaw, range_elevation, t):
    time = t.utc_datetime()
    # receiver position, velocity
    rx_pv = rx.orbit.to_tle().get_orbit_track(time)
    # unit vector tangent to receiver orbit plane (VNB x-axis)
    rx_v_u = rx_pv.velocity.m_per_s / np.linalg.norm(rx_pv.velocity.m_per_s)
    # unit vector normal to receiver orbit plane (VNB y-axis)
    rx_n_u = np.cross(rx_pv.position.m, rx_pv.velocity.m_per_s, 0, 0, -1).T
    rx_n_u = rx_n_u / np.linalg.norm(rx_n_u)
    # unit vector orthogonal to receiver orbit plane (VNB z-axis)
    rx_b_u = rx_pv.position.m / np.linalg.norm(rx_pv.position.m)
    # transmitter position (x_tx), velocity (v_tx)
    tx_pv = tx.orbit.to_tle().get_orbit_track(time)
    # relative position, velocity of transmitter from receiver
    # x_(rx,tx) = x_tx - x_rx; v_(rx,tx) = v_tx - v_rx
    rx_tx_pv = tx_pv - rx_pv
    # tangent point position (m)
    # x_tp = x_tx - x_(rx,tx) . [ x_tx . x_(rx,tx) ] / || x_(rx,tx) ||
    tp_p = tx_pv.position.m - (
        rx_tx_pv.position.m 
        * np.sum(tx_pv.position.m * rx_tx_pv.position.m, axis=0) 
        / np.sum(rx_tx_pv.position.m * rx_tx_pv.position.m, axis=0)
    )
    # intersecting (-1) or parallel (+1) view of tangent point
    tp_sign = np.sign(
        np.sum((tp_p - tx_pv.position.m) * (tp_p - rx_pv.position.m), axis=0)
    )
    # relative transmitter position from receiver in plane normal to receiver orbit
    rx_tx_p_rx_n_plane = rx_tx_pv.position.m - rx_n_u * np.sum(rx_n_u * rx_tx_pv.position.m, axis=0)
    # relative transmitter position from receiver in plane binormal to receiver orbit
    rx_tx_p_rx_t_plane = rx_tx_pv.position.m - rx_b_u * np.sum(rx_b_u * rx_tx_pv.position.m, axis=0)
    # transmitter pitch angle in receiver body-fixed frame
    rx_tx_pitch = np.degrees(
        np.arctan2(
            np.sum(rx_tx_p_rx_n_plane * rx_b_u, axis=0),
            np.sum(rx_tx_p_rx_n_plane * rx_v_u, axis=0),
        )
    )
    # transmitter yaw angle in receiver body-fixed frame
    rx_tx_yaw = np.degrees(
        np.arctan2(
            np.sum(rx_tx_p_rx_t_plane * rx_n_u, axis=0),
            np.sum(rx_tx_p_rx_t_plane * rx_v_u, axis=0),
        )
    )
    # tangent point elevation
    tp_elevation = wgs84.geographic_position_of(
        Geocentric(
            Distance(m=tp_p).au,
            t=t,
        )
    ).elevation.m
    
    is_rising = 1 * (rx_tx_pitch > -90) - 1 * (rx_tx_pitch <= -90)

    # valid if tangent point intersects and yaw angle below maximum
    return is_rising * np.logical_and.reduce((
        tp_sign < 0,
        np.abs(rx_tx_yaw) % (180 - max_yaw) < max_yaw,
        tp_elevation > range_elevation[0],
        tp_elevation < range_elevation[1]
    )).astype(int)

import pandas as pd

ts = pd.date_range(start, start + timedelta(minutes=200), freq=timedelta(seconds=10))
ro = _valid_ro(tx, rx, max_yaw, range_elevation, timescale.from_datetimes(ts))
# ro = [_valid_ro(tx, rx, max_yaw, range_elevation, t) for t in timescale.from_datetimes(ts)]

print(ro)
import matplotlib.pyplot as plt
plt.plot(ts, ro)
plt.show()

In [ ]:
from skyfield.searchlib import find_discrete
from functools import partial

t1 = timescale.from_datetime(start)
t2 = timescale.from_datetime(start + timedelta(minutes=200))
valid_ro_fun = partial(_valid_ro, tx, rx, max_yaw, range_elevation)
valid_ro_fun.step_days=timedelta(minutes=2)/timedelta(days=1)
t, values = find_discrete(t1, t2, valid_ro_fun)

import matplotlib.pyplot as plt
plt.step(np.concatenate(([start], t.utc_datetime(), [start + timedelta(minutes=200)])), np.concatenate(([0], values, [0])), where="post")
plt.show()

In [ ]:
def _tangent_point(tx, rx, t):
    time = t.utc_datetime()
    # receiver position, velocity
    rx_pv = rx.orbit.to_tle().get_orbit_track(time)
    # unit vector normal to receiver orbit plane (VNB y-axis)
    rx_n_u = np.cross(rx_pv.position.m, rx_pv.velocity.m_per_s, 0, 0, -1).T
    rx_n_u = rx_n_u / np.linalg.norm(rx_n_u)

    # transmitter position (x_tx), velocity (v_tx)
    tx_pv = tx.orbit.to_tle().get_orbit_track(time)
    # relative position, velocity of transmitter from receiver
    # x_(rx,tx) = x_tx - x_rx; v_(rx,tx) = v_tx - v_rx
    rx_tx_pv = tx_pv - rx_pv
    # tangent point position (m)
    # x_tp = x_tx - x_(rx,tx) . [ x_tx . x_(rx,tx) ] / || x_(rx,tx) ||
    tp_p = tx_pv.position.m - (
        rx_tx_pv.position.m 
        * np.sum(tx_pv.position.m * rx_tx_pv.position.m, axis=0) 
        / np.sum(rx_tx_pv.position.m * rx_tx_pv.position.m, axis=0)
    )
    # tangent point elevation
    return wgs84.geographic_position_of(
        Geocentric(
            Distance(m=tp_p).au,
            t=t,
        )
    )

ts = pd.date_range(t[0].utc_datetime(), t[1].utc_datetime(), freq=timedelta(seconds=10))

tps = _tangent_point(tx, rx, timescale.from_datetimes(ts))

display(tps)

In [ ]:
def _ro_elevation(target, tx, rx, t):
    # tangent point elevation
    return np.abs(
        target - _tangent_point(tx, rx, t).elevation.m
    )

from skyfield.searchlib import find_minima
from functools import partial

ro_elevation_fun = partial(_ro_elevation, 0, tx, rx)
ro_elevation_fun.step_days=timedelta(minutes=5)/timedelta(days=1)
t, values = find_minima(t1, t2, ro_elevation_fun)

display(t, values)

In [ ]:
from tatc.constants import de421

def _ro_props(tx, rx, t):
    time = t.utc_datetime()
    # receiver position, velocity
    rx_pv = rx.orbit.to_tle().get_orbit_track(time)
    # unit vector tangent to receiver orbit plane (VNB x-axis)
    rx_v_u = rx_pv.velocity.m_per_s / np.linalg.norm(rx_pv.velocity.m_per_s)
    # unit vector normal to receiver orbit plane (VNB y-axis)
    rx_n_u = np.cross(rx_pv.position.m, rx_pv.velocity.m_per_s, 0, 0, -1).T
    rx_n_u = rx_n_u / np.linalg.norm(rx_n_u)
    # unit vector orthogonal to receiver orbit plane (VNB z-axis)
    rx_b_u = rx_pv.position.m / np.linalg.norm(rx_pv.position.m)

    # transmitter position (x_tx), velocity (v_tx)
    tx_pv = tx.orbit.to_tle().get_orbit_track(time)
    # relative position, velocity of transmitter from receiver
    # x_(rx,tx) = x_tx - x_rx; v_(rx,tx) = v_tx - v_rx
    rx_tx_pv = tx_pv - rx_pv
    # relative transmitter position from receiver in plane normal to receiver orbit
    rx_tx_p_rx_n_plane = rx_tx_pv.position.m - rx_n_u * np.sum(rx_n_u * rx_tx_pv.position.m, axis=0)
    # relative transmitter position from receiver in plane binormal to receiver orbit
    rx_tx_p_rx_t_plane = rx_tx_pv.position.m - rx_b_u * np.sum(rx_b_u * rx_tx_pv.position.m, axis=0)
    # transmitter pitch angle in receiver body-fixed frame
    rx_tx_pitch = np.degrees(
        np.arctan2(
            np.sum(rx_tx_p_rx_n_plane * rx_b_u, axis=0),
            np.sum(rx_tx_p_rx_n_plane * rx_v_u, axis=0),
        )
    )
    # transmitter yaw angle in receiver body-fixed frame
    rx_tx_yaw = np.degrees(
        np.arctan2(
            np.sum(rx_tx_p_rx_t_plane * rx_n_u, axis=0),
            np.sum(rx_tx_p_rx_t_plane * rx_v_u, axis=0),
        )
    )
    # tangent point
    tp_geo = _tangent_point(tx, rx, t)

    tx_tle = tx.orbit.to_tle()
    if isinstance(time, datetime):
        tp_tx_azmimuth = (tx_tle.as_skyfield(tx_tle.get_closest_tle_index(time)) - tp_geo).at(t).altaz()[1].degrees
    elif tx_tle.get_tle_count() > 1:
        tp_tx_azmimuth = np.array([
            (
                tx_tle.as_skyfield(tx_tle.get_closest_tle_index(time[i])) 
                - wgs84.latlon(tp_geo.latitude.degrees[i], tp_geo.longitude.degrees[i], tp_geo.elevation.m[i])
            )
            .at(t[i])
            .altaz()[1]
            .degrees
            for i in range(len(time))
        ])
    else:
        tp_tx_azmimuth = (tx_tle.as_skyfield() - tp_geo).at(t).altaz()[1].degrees

    tp_solar_hour = (de421["earth"] + tp_geo).at(t).observe(de421["sun"]).apparent().hadec()[0].hours + 12

    return {
        "tangent_point": tp_geo,
        "rx_tx_pitch": rx_tx_pitch,
        "rx_tx_yaw": rx_tx_yaw,
        "tp_tx_azimuth": tp_tx_azmimuth,
        "tp_solar_hour": tp_solar_hour
    }

_ro_props(tx, rx, t[0])